In [3]:
import math 
import torch 
import random 
# import Value 

In [4]:
class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def tanh(self): 
        x = self.data 
        t = (math.exp(2*x) - 1)/ (math.exp(2*x) + 1)
        out = Value(t, (self, ), "tanh")

        def _backward(): 
            self.grad += (1 - t**2) * out.grad 
        
        out._backward = _backward 

        return out 

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

In [12]:
# let's implement neural network libraray from scratch 

class Neuron: 
    
    def __init__(self, nin:int): # nin -> nr of inputs 
        self.w = [Value(random.uniform(-1, 1)) for x in range(nin)]
        self.b = Value(random.uniform(-1, 1))
        pass

    def __call__(self, x): 
        # tanh( (w * x) + b ) 
        #
        oct = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)  
        out = oct.tanh() 
        return oct  

    def parameters(self): 
        return self.w + [self.b]

class Layer:  # it's just a list of neuron s

    def __init__(self, nin, nout): # nin -> input dimension , nout -> number of neurons in the layer 
        self.neurons = [ Neuron(nin) for _ in range(nout)]

    def __call__(self, x): 
        out = [n(x) for n in self.neurons]
        # print(self.neurons)
        return out[0] if len(out) == 1 else out

    def parameters(self): 
        return [ p for layer in self.neurons for p in layer.paramters()] 

class MLP: 
    
    def __init__(self, nin, nouts): # nouts => sizes of all the layers in the mlp 
        sz = [nin] + nouts 
        self.layers = [ Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x): 
        for layer in self.layers: 
            x = layer(x) 
        
        return x
         
    def parameters(self): 
        return [ p for layer in self.layes for p in layer.paramters()] 





x = [2.0, 3.0 ,-1.0]
n = MLP(3, [4, 4, 1]) # two dimensional neuron, we need three of them 
n(x)

Value(data=-2.4116724020015443, grad=0)

In [11]:
xs = [
    [2.0, 3.0, -1.0], 
    [3.0, -1.0, 0.5], 
    [0.5, 1.0, 1.0], 
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0] 

ypred = [n(x) for x in xs]

# calculate loss 
loss  = sum((yout-ygt)**2 for ygt, yout in zip(ys, ypred))
loss

Value(data=33.98456373541701, grad=0)

AttributeError: 'list' object has no attribute 'parameters'